In [8]:
import fastai
from fastai import *          # Quick access to most common functionality
from fastai.vision import *   # Quick access to computer vision functionality
from fastai.layers import Lambda
from fastai.callbacks import SaveModelCallback
from superres import *
from fast_progress import master_bar, progress_bar
import czifile
import imageio

In [9]:
path = Path('/DATA/WAMRI/salk/uri/MitoTracker_Red_FM_movie_data/')
movie_data = Path('/DATA/WAMRI/salk/uri/movie')

In [10]:
scale=4
n_resblocks=16
n_feats=128
res_scale= 1.
#model = WDSR(scale, n_resblocks, n_feats, res_scale, n_colors=1)
model = torch.load('/home/fredmonroe/wdsr_model_1.3_best').cuda().eval()

In [11]:
def process_file(fn, mb):
    folder = (movie_data/fn.stem)
    folder.mkdir(parents=True, exist_ok=True)
    f = czifile.CziFile(fn)
    
    data = f.asarray().astype(float)
    data += data.min()
    data /= data.max()
    preds = []
    for idx in progress_bar(range(data.shape[3]), parent=mb):
        img_data = data[0,0,0,idx,0,0,:,:,0]
        targ_sz = resize_to(img,96,use_min=True)
        x = tensor(img_data[None]).float().cuda()
        y_pred = model(x).detach()
        y_pred = (torch.clamp(y_pred*3, 0., 1.)*255).cpu().numpy().astype(np.uint8)
        preds.append(y_pred[0])
    all_y = np.concatenate(preds)
    imageio.mimwrite(f'{fn.stem}.mp4', all_y, fps=30, macro_block_size=None)
    imageio.mimwrite(f'{fn.stem}.tiff', all_y)

In [12]:
movie_files = list(path.glob('*230*.czi'))
mb = master_bar(movie_files)
for fn in mb:
    process_file(fn, mb)
    

In [13]:
print('done')

done


In [1]:
learn

NameError: name 'learn' is not defined